## Imports, chargement robuste et normalisation des colonnes

In [ ]:
# --- Imports
import os, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntRangeSlider, FloatSlider, HBox, Output, HTML
from IPython.display import display, clear_output

warnings.filterwarnings("ignore")

CLEAN_DIR = os.path.abspath(os.path.join("..", "data", "clean"))
RAW_DIR   = os.path.abspath(os.path.join("..", "data", "raw"))
clean_fp  = os.path.join(CLEAN_DIR, "dvf_clean.parquet")
raw_txt   = os.path.join(RAW_DIR, "DVF_2025_S1.txt")  # ton fichier texte DVF renommé

def _to_num(s):
    return pd.to_numeric(s.astype(str).str.replace(",", ".", regex=False), errors="coerce")

def load_dvf():
    # 1) Essaye d'abord le parquet propre
    if os.path.exists(clean_fp):
        df = pd.read_parquet(clean_fp)
    else:
        # 2) Sinon, fallback rapide sur le .txt DVF (séparateur |)
        usecols = [
            "Date mutation","Nature mutation","Valeur fonciere",
            "Code postal","Commune","Code departement","Code commune",
            "Type local","Surface reelle bati","Nombre pieces principales"
        ]
        df = pd.read_csv(raw_txt, sep="|", dtype=str, low_memory=False)
        df = df[[c for c in usecols if c in df.columns]].copy()

        # Typage & filtres minimum
        df["Valeur fonciere"] = _to_num(df["Valeur fonciere"])
        df["Surface reelle bati"] = _to_num(df["Surface reelle bati"])
        df["Date mutation"] = pd.to_datetime(df["Date mutation"], errors="coerce")

        # Ventes + Île-de-France
        df = df[df["Nature mutation"].fillna("").str.contains("Vente", case=False, na=False)]
        idf_prefix = ("75","77","78","91","92","93","94","95")
        df = df[df["Code departement"].astype(str).str.startswith(idf_prefix)]

        # prix/m²
        df["prix_m2"] = df["Valeur fonciere"] / df["Surface reelle bati"]
        df = df[(df["Surface reelle bati"] > 8) & (df["prix_m2"].between(100, 30000))]

        # Sauvegarde clean (optionnel)
        os.makedirs(CLEAN_DIR, exist_ok=True)
        df.rename(columns={
            "Date mutation":"date_mutation",
            "Valeur fonciere":"valeur_fonciere",
            "Surface reelle bati":"surface_reelle_bati",
            "Commune":"nom_commune",
            "Code postal":"code_postal",
            "Type local":"type_local"
        }, inplace=True)
        df.to_parquet(clean_fp, index=False)

    # Normalisation des noms attendus par le dashboard
    rename_map = {
        "Date mutation":"date_mutation",
        "Valeur fonciere":"valeur_fonciere",
        "Surface reelle bati":"surface_reelle_bati",
        "Commune":"nom_commune",
        "Code postal":"code_postal",
        "Type local":"type_local"
    }
    df = df.rename(columns=rename_map)

    # Colonnes minimales
    needed = ["date_mutation","valeur_fonciere","surface_reelle_bati","prix_m2","nom_commune"]
    # Si prix_m2 absent (cas parquet externe), on le recalcule
    if "prix_m2" not in df.columns and all(c in df.columns for c in ["valeur_fonciere","surface_reelle_bati"]):
        df["prix_m2"] = df["valeur_fonciere"] / df["surface_reelle_bati"]

    # Nettoyage final minimal
    df = df.dropna(subset=["surface_reelle_bati","valeur_fonciere","prix_m2","nom_commune"]).copy()
    df = df[(df["surface_reelle_bati"] > 8) & (df["valeur_fonciere"] > 1000) & (df["prix_m2"].between(100, 30000))]
    df["annee"] = pd.to_datetime(df["date_mutation"], errors="coerce").dt.year
    return df[needed + ["annee"]]

df = load_dvf()
len(df), df.columns.tolist()


## Widgets 5 (surface) & 6 (loyer €/m²)

In [ ]:
w_surface = IntRangeSlider(
    value=(20, 80),
    min=10, max=200, step=1,
    description="Surface (m²)",
    continuous_update=False
)

# Valeur par défaut réaliste pour IDF (tu peux ajuster)
w_loyer = FloatSlider(
    value=22.0, min=5.0, max=45.0, step=0.5,
    readout_format=".1f",
    description="Loyer €/m²",
    continuous_update=False
)

out = Output()
display(HBox([w_surface, w_loyer]), out)


## Fonctions + rendu (tableau Top communes + histogramme)

In [ ]:
def filter_by_surface(d: pd.DataFrame, s_range):
    smin, smax = s_range
    return d[d["surface_reelle_bati"].between(smin, smax)]

def compute_yield(d: pd.DataFrame, loyer_m2: float) -> pd.DataFrame:
    res = d.copy()
    # Revenu annuel estimé (brut)
    res["revenu_annuel"] = loyer_m2 * res["surface_reelle_bati"] * 12.0
    res["yield_brut"] = res["revenu_annuel"] / res["valeur_fonciere"]
    return res

def render(_=None):
    with out:
        clear_output(wait=True)
        d = filter_by_surface(df, w_surface.value)
        d = compute_yield(d, w_loyer.value)

        # Classement communes par rendement médian
        g = (d.groupby("nom_commune", dropna=False)
               .agg(nb=("prix_m2","count"),
                    prix_m2_med=("prix_m2","median"),
                    surf_med=("surface_reelle_bati","median"),
                    yield_brut_med=("yield_brut","median"))
               .sort_values("yield_brut_med", ascending=False)
               .head(15)
               .reset_index())

        display(HTML("<h4>Top 15 communes (rendement brut médian)</h4>"))
        display(g)

        # Histogramme des rendements (capé à 25% pour lisibilité)
        plt.figure()
        d["yield_brut"].dropna().clip(upper=0.25).plot(kind="hist", bins=40)
        plt.xlabel("Rendement brut")
        plt.ylabel("Nombre de ventes")
        plt.title("Distribution des rendements (Surface + Loyer €/m²)")
        plt.show()

w_surface.observe(render, "value")
w_loyer.observe(render, "value")
render()
